# 272B Homework 4: TA Version
---
## PART 2
---

## Imports, Constants, Dataset loading

In [1]:
!pip install -q transformers
!pip install -q datasets

     |████████████████████████████████| 3.8 MB 13.1 MB/s 
     |████████████████████████████████| 6.5 MB 47.9 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 895 kB 47.0 MB/s 
     |████████████████████████████████| 596 kB 46.1 MB/s 
     |████████████████████████████████| 312 kB 25.5 MB/s 
     |████████████████████████████████| 1.1 MB 62.2 MB/s 
     |████████████████████████████████| 134 kB 71.4 MB/s 
     |████████████████████████████████| 212 kB 71.5 MB/s 
     |████████████████████████████████| 127 kB 74.1 MB/s 
     |████████████████████████████████| 144 kB 72.0 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████████████████████| 271 kB 72.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which 

In [2]:
# TensorFlow Imports
import tensorflow as tf
from tensorflow.keras import layers, models, utils, preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.strings import lower, regex_replace
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
import numpy as np
import datasets
import os

# These might help
int2language= {0: 'csharp', 1: 'java', 2 :'javascript', 3: 'python'}
language2int= {'csharp': 0, 'java': 1, 'javascript': 2, 'python': 3}

### Parameters and Helper functions

In [3]:
# TODO: Set parameters you want for part 2 (or hard code later, but this might be easier)
# TODO make sure the validation split size matches your LSTM 
# The epoch is set to 1, for debugging. Be sure to increase this later.
validation_split = 0.2
batch_size = 32
epochs = 3
label_mode = 'int'
vocab_size = 10000
max_text_length = 250

### Problem (1) Load the dataset, and split into sets. 
(a-b) Code. Download dataset. split into train, validation, and test. This should be the same as last week.

(c) Code. Convert to ```datasets.DatasetDict``` format using the given conversion function. Instructions are in the pdf.

In [4]:
data_url = "https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz"
# TODO YOUR FOLDER HERE
data_dir = '/content/hw4'

# the rest is done for you
data_cache = utils.get_file(
    'stack_overflow_16k.tar.gz',
    data_url,
    untar = True,
    cache_dir = '',
    cache_subdir = data_dir
)

dataset_args = dict(
    seed=0,
    batch_size=batch_size,
    label_mode=label_mode
)

data_train = preprocessing.text_dataset_from_directory(
    data_dir + "/train", 
    validation_split = validation_split, 
    subset = "training", 
    **dataset_args
)

data_val = preprocessing.text_dataset_from_directory(
    data_dir + "/train", 
    validation_split = validation_split, 
    subset = "validation", 
    **dataset_args
)

data_test = preprocessing.text_dataset_from_directory(data_dir + "/test", label_mode = label_mode)

6062080/6053168 [==============================] - 0s 0us/step
Found 8000 files belonging to 4 classes.
Using 6400 files for training.
Found 8000 files belonging to 4 classes.
Using 1600 files for validation.
Found 8000 files belonging to 4 classes.


In [5]:
# This converts to hugginface datasets for you.
def tfds_to_hugds(tfds, split_name):
    x, y = [], np.array([], dtype=int)
    for batchx, batchy in tfds:
        x.extend([t.numpy().decode().replace('"', '') for t in batchx])
        y = np.append(y, batchy.numpy())
        
    return datasets.Dataset.from_dict({'text': x, 'labels': y}, split=split_name)

# TODO follow the pdf instructions to load a DatasetDict (lets you preprocess on all 3 splits)
huggingface_data = datasets.DatasetDict(
    train = tfds_to_hugds(data_train, 'train'),
    validation = tfds_to_hugds(data_val, 'validation'),
    test = tfds_to_hugds(data_test, 'test')
)

## All Students - Main Assignment

---
### Problem (2) Vectorize the Text
 (a) Use ```AutoTokenizer``` from transformers to load a BERT tokenizer. We will use
```distilbert-base-uncased```.

(b) Take a training sample text, and pass it through the ```Tokenizer```. Compare it to
passing it through your ```TextVectorization```layer. (Quickly load/adapt a ```TextVectorization``` layer, this code is the same as last week.) Describe the differences in the output.
You will have to explore a bit here using built-ins like ```type()``` or ```.keys()```.
> The output of `Tokenizer` has a single-row `input_ids` vector (each vocabulary is mapped to a token, i.e., an single element of array) and a `attention_mask` vector, whereas the output of `TextVectorization` is a 2D array (each vocabulary is mapped to a vector, i.e., a row vector of array).

(c) Call ```tokenizer.decode()``` and ```tokenizer.convert_ids_to_tokens()``` on input_ids from the tokenizer output above. What do you notice?
> `tokenizer.decode` converts the token vector to the original text example, whereas `tokenizer.convert_ids_to_tokens` converts the `input_ids` of `tokenized_example` and lists all the tokens out.

In [6]:
import string
import re

# TODO Copy your standardization function and load a TextVectorization layer just like last week.
def custom_standardization(inputs):
    lowercase = lower(inputs)
    stripped_html = regex_replace(lowercase, '<br />', ' ')

    return regex_replace(stripped_html, f"[{re.escape(string.punctuation)}]", "")

encoder = TextVectorization(max_tokens = vocab_size, output_sequence_length = max_text_length, standardize = custom_standardization)
encoder.adapt(data_train.map(lambda question, language: question))


# TODO load a BERT Tokenizer. Be sure to indicate the maximum sentence length.
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", max_length=max_text_length)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [7]:
# TODO pick a sample from the data, get the tokenized version
for text_batch, label_batch in data_train.take(1):
    text = text_batch.numpy()[0]
    label = label_batch.numpy()[0]

sample = text.decode('utf-8')
tokenized_sample = tokenizer(sample)
vectorized_sample = encoder(sample)

# TODO use print statements and answer above to describe the difference in these two methods for 
# changing text to numbers. Look at things like .shape, .keys(), len, as well as actual printouts of values.
print("My comparisons here: ")
print("tokenized_sample ", tokenized_sample)
print("vectorized_sample ", vectorized_sample)

Token indices sequence length is longer than the specified maximum sequence length for this model (542 > 512). Running this sequence through the model will result in indexing errors


My comparisons here: 
tokenized_sample  {'input_ids': [101, 1000, 2129, 2000, 3229, 1996, 6013, 2096, 2478, 9185, 1029, 1045, 2031, 1037, 2465, 2008, 7077, 2083, 2019, 6013, 1005, 1055, 2266, 2012, 2448, 1011, 2051, 1012, 1012, 1012, 2917, 2003, 1037, 2465, 2008, 5383, 2367, 2492, 2007, 2367, 2828, 1012, 11412, 18209, 5397, 2003, 2028, 1997, 1996, 2828, 1045, 2359, 2000, 7077, 2083, 1012, 1012, 1012, 2270, 2465, 1037, 1063, 1012, 1012, 2270, 10763, 11412, 18209, 5397, 1056, 1035, 1038, 2290, 1027, 2047, 11412, 18209, 5397, 1006, 1000, 1000, 1038, 2290, 1012, 1052, 3070, 1000, 1000, 1007, 1025, 1012, 2270, 10763, 11412, 18209, 5397, 1056, 1035, 15934, 2015, 1027, 2047, 11412, 18209, 5397, 1006, 1000, 1000, 15934, 2015, 1012, 1052, 3070, 1000, 1000, 1007, 1025, 1012, 2270, 10763, 11412, 18209, 5397, 1056, 1035, 8154, 1027, 2047, 11412, 18209, 5397, 1006, 1000, 1000, 8154, 1012, 1052, 3070, 1000, 1000, 1007, 1025, 1012, 1012, 2270, 10763, 5164, 2477, 1035, 2008, 1035, 2323, 1035, 2025, 10

In [8]:
print("Original text ", sample)
print("tokenizer.decode ", tokenizer.decode(tokenized_sample['input_ids']))
print("tokenizer.convert_ids_to_tokens ", tokenizer.convert_ids_to_tokens(tokenized_sample['input_ids']))

Original text  "how to access the instance while using reflection? i have a class that loop through an instance's member at run-time...below is a class that contain different field with different type. assettexture is one of the type i wanted to loop through...public class a {..    public static assettexture t_bg          = new assettexture(""bg.png"");.    public static assettexture t_reels       = new assettexture(""reels.png"");.    public static assettexture t_logo        = new assettexture(""logo.png"");..    public static string things_that_should_not_inside_the_loop;..}...below is the sample code of assettexture class. it only contain a name...class assettexture {..    private string name;..    public assettexture(string name){.        this.name=name;.    }..}...below is the method i use to loop through all field at runtime. it is successful. but i have try all available method of field. it doesn't have the method to get the member...public class manager {..    public manager(){

In [9]:
# TODO define this function to call the tokenizer with the proper arguments to make our training dataset
#def tokenize(texts):
#    return tokenizer(texts['text'], truncation=True, padding='max_length')

tokenized = huggingface_data.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length'), batched=True)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

---
### Problem (3) Conceptual Break: Read this [blog on transformers and self-attention](https://towardsdatascience.com/transformers-89034557de14). 
You may also read the original paper, linked in the Grad section below, if that is more helpful.

(a) Explain 2 disadvantages of RNNs (LSTM) in NLP tasks. You will find this information from lecture/sources outside the blog.

1.   It has gradient exploding problems.
2.   Training a RNN model is tough (e.g., more time consuming, variant losses).



(b) Reprinted for convenience is the self-attention mechanism. The letters correspond to "Query", "Key" and "Value" matrices. $d_k$ is the hidden dimension size, but is not super important for us to look at right now.
$$ A = softmax(\frac{Q K^T }{\sqrt{d_k}}) V$$
> * Explain, in your own words , the purpose of Query/Key/Value vectors in self attention.
>>  
    * Query (Q): current word
    * Key (K): used to index the value vectors uniquely
    * Value (V): information of the input word


> * Explain how self-attention fixes the 2 main disadvantages of LSTM.
>> The division of $QK^T$ by $\sqrt{d_k}$, where $d_k$ is the dimension of $K$, can benefit gradient flow to avoid gradient explosions. Applying the softmax function can preserve high probability scores and remove the rest to reduce calculation intensity.

(c) A LSTM's structure inherently means that current cell will only have information about past words (from the hidden state passed along). This means we get both (1) information in a causal manner, and (2) positional information by the order in which the words are processed.
> * Explain how transformers add position information to the input words. See **Beast #3** in the blog.
>> The transformer injects *positional encoding* to individual input embeddings. *Positional encoding* follows some periodic functions which the transformer learn from and is able to determine the position of individual word w.r.t each other.
> * Explain how transformers make sure the decoder stack acts in a causal manner as well (We don't want it to "look into the future" at words that haven't been generated yet). See **Beast #4**  in the blog.
>> The combination of*positional encoding* and *word embedding* in the decoder stack ensures the predictions for a position can only depend on the already known outputs from the previous positions.

---
### Problem (4) Load the BERT model and prepare for training
(a) Load distilbert-base-uncased using AutoModelForSequenceClassification. Make sure to indicate how many classes!

(b) Explain the difference between AutoModelForSequenceClassification and  AutoModel in view of "foundation models"
>
    * AutoModel directly creates a class of the relevant architecture.
    * AutoModelForSequenceClassification 

(c) Set TrainingArguments. (Some) Initial arguments are given for you, which
should be adequate. The number of epochs may change when you look at the training time.

(d) Load a Hugginface Trainer to use the tokenized train and validation data.

In [10]:
# TODO load pretrained bert model 'distilbert-base-uncased' as AutoModelForSequenceClassification
# Remember we have 4 classes.
bert = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=4)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [11]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding, TrainerCallback
# This collator just speeds things up in the background
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Define arguments
training_args = TrainingArguments(
    # TODO: read, but you do not need to change
    weight_decay=0.01,
    optim='adamw_torch',
    evaluation_strategy = 'epoch',
    learning_rate=2e-5,
    # TODO: read, you may change if you want
    output_dir="./results",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
)

trainer = Trainer(
    model=bert,
    args=training_args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

---
### Problem (5) Train ("fine-tune") the BERT model!
(a) Train it - ```trainer.train()```. That's it!

(b) Comment on the training time compared to the LSTM. Any thoughts on why?
> The LSTM in Part 1 took less time in comparison with BERT in this homework. I think this is mainly due to the difference of complexities of the two models.

(c) HuggingFace's interface makes the training a one line call with no arguments instead of TF's (define loss) --> (compile) --> (fit). Where in our process would we have set all the hyper-parameters?
> The hyper-parameters are set in the compile stage, where the optimizer is specified.

In [12]:
train_output = trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 6400
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 600


Epoch,Training Loss,Validation Loss
1,No log,0.528936
2,No log,0.422081
3,0.537600,0.417658


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 32
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/specia

--- 
### Problem (6) Evaluate
(a) Report the test set accuracy. You can get the predicted logits from ```trainer.predict(...)```. How does this compare to your best LSTM accuracy?

(b) Take the following questions as input, what are the predicted tags? A function is provided to correctly get the logits for you, since after
training some of the the model will be on the gpu, but the text you make will be on the
cpu.
>i. "how do I extract keys from a dict into a list?"
>
>ii. "debug public static void main(string[] args) ..."

Do you agree?
> The prediction for the first tag is python, and the prediction for the second tag is java. Both predictions make sense since the keywords or the syntaxes matches the type of language.

(c) What is the most annoying aspect about using/training/evaluating BERT? For
our problem, is this annoyance worth the performance?
> It could take an hour just to train for 5 epochs with BERT. However, since the accuracy of the test results is higher than that of Part 1's test results by 12%, I think it's a pretty decent tradeoff.

In [13]:
# This will get the predictions on the set, it will take a while though
test_results = trainer.predict(tokenized['test'])

The following columns in the test set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 8000
  Batch size = 32


In [14]:
# Sklearns function could help with accuracies
from sklearn.metrics import accuracy_score

# TODO convert the logits into class predictions and compute accuracy
test_predictions = np.argmax(test_results[0], 1)
accuracy = np.sum(test_predictions == test_results[1]) / len(test_predictions)

# TODO display results
print(f"Test accuracy: {accuracy}, {np.sum(test_predictions == test_results[1])}/{len(test_predictions)}")


Test accuracy: 0.848, 6784/8000


In [19]:
# TODO try out those sentences again
trials = ["how do I extract keys from a dict into a list?", "debug public static void main(string[] args) ..."]

# this function will help deal with tensors being on different devices
def try_bert(text):
    bert_input = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True)
    bert_output = bert(**bert_input.to('cuda:0'))
    logits = bert_output['logits'].to('cpu').detach().numpy()
    return logits

# TODO display the outputs
for i, trial in enumerate(trials): 
    logits = try_bert(trial)
    index = np.argmax(logits, 1)
    print(f"{trial}: {int2language[index[0]]}\n")

how do I extract keys from a dict into a list?: python

debug public static void main(string[] args) ...: java



---
## Grad Students / EC

(a) Read the [original transformers paper](https://arxiv.org/abs/1706.03762)

(b) Explain the main computational limitation of transformers/BERT, especially in comparison with LSTM. Be thorough.
> Transformers can only deal with fixed-length text strings. The text has to be splited into a certain number of segments before being fed into the model.

(c)  Explain the purpose of the $\sqrt{d_k}$ term in the self attention equation.
> It is a scaling factor which counteracts the growth of dot products $QK^T$ in magnitude that may cause the softmax functionbe pushed into regions with extremely small gradients.

(d) Instead of pure self-attention, the authors used multi-head-attention. Explain the purpose of this mechanism i) intuitively, and ii) in terms of computational complexity.
> Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions. The total computational cost is similar to that of the single-head attention with full dimensionality due to the reduced dimension of each head, where $head_i=Attention(Q{W_i}^Q, K{W_i}^K, V{W_i}^V)$.

(e) It's not the easiest to see from the paper, but their diagram shows a "add and norm" layer within each transformer layer. Explain the "add" component to the best of your understanding. (Hint - is something here similar to Recurrent networks? Residual networks?). If you are unsure, that is OK. Try to find some intuition and defend your stance.
> The "add" component of the add and norm layer is a residual connection, which adds the output of the layer with the input: $F(x)+x$. This component is meant to deal with the vanishing gradient problem.